
# Parsl Fest 2025: Parallel Notebook Testing in QMCPy with Parsl


Joshua Herman and Sou-Cheng Choi, QMCPy Developers

Aug 28 -- 29, 2025

**Requirements**:

* QMCPy: `pip install qmcpy==1.6.1.2`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

In [1]:
import qmcpy
import sys
import os
import time

In [2]:
start_time = time.time()
!(cd .. && make booktests)
sequential_time = time.time() - start_time


Notebook tests
cd test/booktests/ && \
PYTHONWARNINGS="ignore::UserWarning,ignore::DeprecationWarning,ignore::FutureWarning,ignore::ImportWarning" \
python -W ignore -m coverage run --append --source=../../qmcpy/ -m unittest discover -s . -p "*.py" -v 1>/dev/null && \
rm -fr *.eps *.jpg *.pdf *.part && \
cd ../..
test_argonne_talk_2023_figures_notebook (tb_Argonne_2023_Talk_Figures.NotebookTests) ... skipped 'Runtime error'
test_mcqmc2022_article_figures_notebook (tb_MCQMC2022_Article_Figures.NotebookTests) ... skipped 'Runtime error'
test_mc_vs_qmc_notebook (tb_MC_vs_QMC.NotebookTests) ... ok
test_pricing_asian_options_notebook (tb_PricingAsianOptions.NotebookTests) ... ok
test_purdue_talk_figures_notebook (tb_Purdue_Talk_Figures.NotebookTests) ... skipped 'Runtime error'
test_asian_option_mlqmc_notebook (tb_asian_option_mlqmc.NotebookTests) ... ok
test_control_variates_notebook (tb_control_variates.NotebookTests) ... ok
test_dakota_genz_notebook (tb_dakota_genz.NotebookTests) ... ok

## 2. Parsl

1. Install and Configure Parsl
2. Run the tests in Parallel with Parsl

### 2.1 Install and Configure Parsl

In [3]:
import parsl as pl
from parsl.configs.htex_local import config

config.max_workers = 8  
if not pl.config:
    pl.load(config)
    print("Parsl loaded with memory limits")
else:
    print("Parsl already configured")

Parsl already configured


### 2.2 Create a Parsl Test Runner

In [4]:
# Ensure the path to the booktests directory is included
sys.path.append(os.path.join(os.getcwd(), '..', 'test', 'booktests'))

In [5]:
import parsl_test_runner
import inspect

# See only functions
print("Functions:")
functions = inspect.getmembers(parsl_test_runner, inspect.isfunction)
for name, func in functions:
    print(f"- {name}")

print("\n" + "="*50)

# Get help on specific function
print("Help for execute_parallel_tests:")
help(parsl_test_runner.execute_parallel_tests)

Functions:
- bash_app
- execute_parallel_tests
- generate_summary_report
- main

Help for execute_parallel_tests:
Help on function execute_parallel_tests in module parsl_test_runner:

execute_parallel_tests()
    Execute all testbook tests in parallel using Parsl



### 2.3 Run the Notebook in Parallel with Parsl

In [6]:
start_time2 = time.time()
!(cd .. && make booktests-parallel)
parallel_time = time.time() - start_time2


Booktests
cd test/booktests/ && \
PYTHONWARNINGS="ignore::UserWarning,ignore::DeprecationWarning,ignore::FutureWarning,ignore::ImportWarning" \
python parsl_test_runner.py  1>/dev/null && \
rm -fr *.eps *.jpg *.pdf *.part && rm -fr logs && rm -fr runinfo && \
cd ../.. 


In [7]:
speedup = sequential_time / parallel_time
print(f"Sequential time:  {sequential_time:.2f} seconds")
print(f"Parallel time:    {parallel_time:.2f} seconds")
print(f"Speedup by Parsl: {speedup:.2f}")

Sequential time:  1689.76 seconds
Parallel time:    781.21 seconds
Speedup by Parsl: 2.16


In [8]:
!uname -a
!nproc --all
!awk '/MemTotal/ {printf "%.2f GB\n", $2/1024/1024}' /proc/meminfo

Linux codespaces-e74c2d 6.8.0-1030-azure #35~22.04.1-Ubuntu SMP Mon May 26 18:08:30 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
16
62.80 GB
